# Submission Notebook

This is a rule based approach.

## Part I : Data Preparation

Package installation

In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
!pip install pickle
!pip install json

Imports

In [ ]:
import pickle  # Needed for us to parse the building definition files
import json    # Needed to save the files
import time    # Allows us to provide a time estimate of the frugality of our approach

Loading resources

In [ ]:
"""
The buildings mentionned below are specific to the hackathon and are not available in this repo.
You can replace them with any MicroGrid object generated from pymgrid
"""

buildings = []



for building_file in ['building_1.pkl','building_2.pkl','building_3.pkl']:
    with open(building_file, 'rb') as f:   # We load in our building definition files in memory
        building = pickle.load(f)          # We parse their content to extract their useful data
        building.train_test_split()        # The data is split into a training and testing set
        buildings.append(building)

## Part II : Definition of the strategy

This helper function helps us return the appropriate control dictionary depending on the overall action we want to perform.

In [ ]:
def action_to_control_dict(building_state, action):
    '''
    Arguments :
    - building_state : the building object, from which we extract informations
    - action : a string describing the action
    Returns the control dictionary corresponding to the action
    '''

    ### First, retreive the important informations

    # We retrieve the power output coming from the solar arrays
    pv = building_state.pv
    # And the current load that is imposed on it
    load = building_state.load

    # We want to use up all the energy possible from the solar arrays,
    # hence what remains to be produced is the difference between the
    # load and their output
    net_load = load - pv

    # We check whether there is an outage or not
    status = building_state.grid.status

    # We retrieve data about the battery from those properties to be able to
    # interact with it
    capa_to_charge = building_state.battery.capa_to_charge
    p_charge_max = building_state.battery.p_charge_max 
    capa_to_discharge = building_state.battery.capa_to_discharge
    p_discharge_max = building_state.battery.p_discharge_max                                                                     

    # 0 <= what we can put in the battery <= -net load
    p_charge_pv = max(0, min(-net_load, capa_to_charge, p_charge_max))
    # Same reasoning for what we can give back    
    p_charge_grid = max(0, min( capa_to_charge, p_charge_max))
    # Same reasoning for discharging the battery
    p_discharge = max(0, min(net_load, capa_to_discharge, p_discharge_max))
    
    # Relevant for building 3 : if the building has a generator, we retrieve its capacity
    if hasattr(building_state, 'genset'):                                                   
        capa_to_genset = building_state.genset.rated_power * building_state.genset.p_max
    else:
        capa_to_genset = 0
    p_genset = max(0, min(net_load, capa_to_genset))

    ### Second, Define the discrete actions used in our strategy

    # In this case we charge the battery and use the solar arrays
    if action == 'pv':
        control_dict = {'pv_consummed': min(pv, load),
                        'battery_charge': p_charge_pv,
                        'battery_discharge': 0,
                        'grid_import': 0,
                        'grid_export': max(0, pv - min(pv, load) - p_charge_pv),
                        'genset': 0
                        }

    # We discharge the battery to meet the building's energy requirement, and import from grid if needed
    elif action == 'discharge':
        control_dict = {'pv_consummed': min(pv, load),
                        'battery_charge': 0,
                        'battery_discharge': p_discharge,
                        'grid_import': max(0, load - min(pv, load) - p_discharge),
                        'grid_export': 0,
                        'genset': 0
                        }

    # We make use of the grid to meet the buildin't energy requirement, and don't interact with the battery
    elif action == 'grid':
        control_dict = {'pv_consummed': min(pv, load),
                        'battery_charge': 0,
                        'battery_discharge': 0,
                        'grid_import': max(0, net_load),
                        'grid_export': 0,
                        'genset': 0
                        }
    
    # Charge the battery using the grid, and if can't, import or export the excedent
    elif action == 'charge':
        load = load + p_charge_grid
        control_dict = {'pv_consummed': min(pv, load),
                        'battery_charge': p_charge_grid,
                        'battery_discharge': 0,
                        'grid_import': max(0, load - min(pv, load)),
                        'grid_export': max(0, pv - min(pv, load) - p_charge_grid) ,
                        'genset': 0
                        }

    # Use the genset and battery to meet the buildin't energy requirement, completely off-grid
    elif action == 'genset':
        control_dict = {'pv_consummed': min(pv, load),
                        'battery_charge': 0,
                        'battery_discharge': p_discharge,
                        'grid_import': 0,
                        'grid_export': 0,
                        'genset': max(0, load - min(pv, load) - p_discharge),
                        }

    # If the control dict is not available, the action is not a valid one
    if control_dict is None:
        print('Error : the action does not exist')

    # Getting what discrete actions we could perform on the building
    control_dict_keys = building_state.get_control_dict()

    # We add the missing keys to the control dictionary
    for k in control_dict_keys:
        if k not in control_dict:
            control_dict[k] = 0

    return control_dict

Implementation of our algorithm

In [ ]:
def rule_based_strategy(building):

    # We retrieve up-to-date data about the building
    building_data = building.get_updated_values()

    # Initialises a counter to keep track of the algorithm's efficiency
    total_building_cost = 0

    while not building.done:

        # We get data from the building's state
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']
        battery_soc = building_data['battery_soc']
        grid_co2 = building_data['grid_co2']
        grid_price_import = building_data['grid_price_import']
        grid_price_export = building_data['grid_price_export']

        # 0.3 is the price below which electricity is considered cheap
        # This value has been selected because the price of electricity can take 3 values :
        # 0.19, 0.29 and 0.65. The third price is obvisouly "big" and the other ones "small"
        elec_is_cheap = grid_price_import < 0.3                                             

        # Is the battery empty ? 
        # To account for floating point errors, eps=1e5 is added      
        bat_is_not_empty = battery_soc > building.battery.soc_min + 1e-5

        # We check whether there is an outage or not. 
        # If there is, we know that we are in building 3 and therfore we have a genset
        use_genset = (building_data['grid_status']==0)

        # We check whether there is fuel generator or not
        has_gen_set = hasattr(building, 'genset')

        # Computing what we can acceptably get from the battery's discharge
        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        # Conversely, he amound up to which we can potentially charge the battery
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))
        
        # The 6 following conditions do not overlap

        # 1
        # In this case we try to get the maximum energy from the battery since
        # electricity is expensive, but we can still get it from the grid if
        # absolutely needed
        if load >= pv and not elec_is_cheap and not use_genset:                                                
            control_dict = action_to_control_dict(building, 'discharge')
        
        # 2
        # Here, if electricity is cheap, we just rely on the grid,
        # the battery is not needed
        if load >= pv and elec_is_cheap and bat_is_not_empty and not has_gen_set:
            control_dict = action_to_control_dict(building, 'grid')               
        
        # 3
        # The battery is empty, we can charge it with the grid and
        # import or export the excedent for the building
        if load >= pv and elec_is_cheap and not bat_is_not_empty and not has_gen_set:                           
            control_dict = action_to_control_dict(building, 'charge')                                         

        # 4
        # The battery is empty, but because there is a risk of outage,
        # it is better to keep the battery full
        if load >= pv and elec_is_cheap and has_gen_set and not use_genset:
            control_dict = action_to_control_dict(building, 'charge')
        
        # 5
        # We can rely fully on the solar arrays to power the building
        if pv > load:
            control_dict = action_to_control_dict(building, 'pv')
        
        # 6
        # The grid is unavailable, so we use the genset to power the building
        if load >= pv and use_genset:
            control_dict = action_to_control_dict(building, 'genset')

        

        # We apply out choice for this step
        building_data = building.run(control_dict)
        # We add on to the total cost                                              
        total_building_cost += building.get_cost()                                      
    
    return total_building_cost

## Part III : Evaluation the strategy

Run of the rules on the Test environment

In [ ]:
eval_start = time.process_time()

total_building_costs = []

for building in buildings:

    building.reset(testing = True)

    total_building_cost = rule_based_strategy(building)
    total_building_costs.append(total_building_cost)

eval_end = time.process_time()

In [ ]:
total_cost_building_1 = total_building_costs[0]
total_cost_building_2 = total_building_costs[1]
total_cost_building_3 = total_building_costs[2]

In [ ]:
frugality = eval_end - eval_start

 Store & Export Results in JSON format

In [ ]:
final_results = {
    "building_1_performance" : total_cost_building_1,
    "building_2_performance" : total_cost_building_2,
    "building_3_performance" : total_cost_building_3,
    "frugality" : frugality,
}
print(final_results)